In [7]:
# Importing libraries
import pandas as pd

In [8]:
# Loading dataset
customer_df = pd.read_csv("../data/processed/customer_level_data.csv")
customer_df.head()

,customerid,first_purchase,last_purchase,total_orders,total_revenue,tenure_days
0,12346.0,2011-01-18 10:01:00,2011-01-18 10:01:00,1,77183.60,0
1,12347.0,2010-12-07 14:57:00,2011-12-07 15:52:00,7,4310.00,365
2,12348.0,2010-12-16 19:09:00,2011-09-25 13:13:00,4,1797.24,282
3,12349.0,2011-11-21 09:51:00,2011-11-21 09:51:00,1,1757.55,0
4,12350.0,2011-02-02 16:01:00,2011-02-02 16:01:00,1,334.40,0


In [9]:
customer_df["customer_type"] = customer_df["total_orders"].apply(
    lambda x: "Returning" if x > 1 else "One-time"
)

In [10]:
customer_df["customer_type"].value_counts(normalize=True) * 100

customer_type
Returning    65.583218
One-time     34.416782
Name: proportion, dtype: float64

In [11]:
# Calculating the proportion of returning customers

repeat_rate = (
    customer_df[
        customer_df["total_orders"] > 1
    ].shape[0] / customer_df.shape[0]
) * 100

repeat_rate

65.58321807284463

Cohort Analysis

In [12]:
df = pd.read_csv("../data/processed/clean_retail_data.csv")
df["invoicedate"] = pd.to_datetime(df["invoicedate"])

df["purchase_month"] = df["invoicedate"].dt.to_period("M")

In [13]:
df["cohort_month"] = df.groupby("customerid")["purchase_month"].transform("min")

In [14]:
def cohort_index(row):
    return (
        (row["purchase_month"].year - row["cohort_month"].year) * 12
        + (row["purchase_month"].month - row["cohort_month"].month)
        + 1
    )


df["cohort_index"] = df.apply(cohort_index, axis=1)

In [15]:
cohort_data = (
    df.groupby(["cohort_month", "cohort_index"])["customerid"].nunique().reset_index()
)

In [16]:
customer_df.to_csv("../data/processed/customer_retention_summary.csv", index=False)
cohort_data.to_csv("../data/processed/cohort_table.csv", index=False)